# DatasetName Pre-processing
## Team 5 - Connor, John, Libby, & Natalie

This file contains the code used to preprocess and clean our selected dataset. The final step will be outputting a cleaned and processed dataset as a new .csv file.

#### Setup

In [ ]:
#Install Required Libraries (Only needs to run once)
%pip install -q -r requirements.txt

In [ ]:
#import libraries
import pandas as pd

#### Import Data

In [ ]:
#Paths
DATASET1PATH = ""
DATASET2PATH = ""

In [ ]:
#Import Dataset
df1 = pd.read_csv(DATASET1PATH)
df2 = pd.read_csv(DATASET2PATH)

datasets = [df1, df2]

#### Data Cleaning

In [ ]:
#Row wise Null counts
